In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import glob
import json
from haversine import haversine

pd.options.display.max_columns=999
os.getcwd()
import gc
# Get teenagers and mothers only

# Get all of the POI places

In [2]:
# requested of Walmart

In [3]:
df_brand_info=pd.read_csv("/home/jian/Docs/POI/places_20200406/part-00000-tid-3169497161568699755-0a71d798-e636-4e2f-949e-02e4669f577c-43-c000.csv.gz",
                          compression="gzip",dtype=str,sep="|")
df_brand_info_wmt=df_brand_info[df_brand_info['brand_name'].apply(lambda x: "walmart" in x.lower())]
list_brand_names=df_brand_info_wmt['brand_name'].unique().tolist()
del df_brand_info
del df_brand_info_wmt
gc.collect()

0

In [4]:
list_files_POI=glob.glob("/home/jian/Docs/POI/places_20200406/*")
list_files_POI=[x for x in list_files_POI if "poi" in os.path.basename(x).lower()]

In [5]:
list_brand_names

['walmart', 'walmart inc']

In [6]:
df_walmart_location=pd.DataFrame()
for file in list_files_POI:
    df=pd.read_csv(file,dtype=str,compression="gzip",sep=",")
    for brand_name in list_brand_names:
        df_b=df[df['brands'].apply(lambda x: brand_name in str(x).lower())]
        df_walmart_location=df_walmart_location.append(df_b)
print(df_walmart_location.shape,df_walmart_location['safegraph_place_id'].nunique())
print(df_walmart_location['brands'].unique().tolist())
print(df_walmart_location['location_name'].unique().tolist())

(4693, 22) 4693
['Walmart']
['Walmart', 'Walmart Supercenter', 'Wal Mart Supercenter', 'Alice Walmart Center', 'Plainview Walmart Center', 'Morris Super Walmart']


In [7]:
df_walmart_location=df_walmart_location[['safegraph_place_id','location_name','brands','street_address','city','region','postal_code','latitude','longitude']]
df_walmart_location['latitude']=df_walmart_location['latitude'].astype(float)
df_walmart_location['longitude']=df_walmart_location['longitude'].astype(float)
df_walmart_location.head(2)

,safegraph_place_id,location_name,brands,street_address,city,region,postal_code,latitude,longitude
921,sg:43f385d5026e4fffb55bbd42f0a91302,Walmart,Walmart,2400 Veterans Memorial Pkwy,Orange City,FL,32763,28.913804,-81.278884
1045,sg:4a8dc2497b1441ca8292e9ed302dd966,Walmart,Walmart,201 Gateway Blvd,Rock Springs,WY,82901,41.580524,-109.251274


In [8]:
df_walmart_location_stores_to_Joann=df_walmart_location.copy()
df_walmart_location_stores_to_Joann=df_walmart_location_stores_to_Joann.rename(columns={"safegraph_place_id":"poi_place_id"})
df_walmart_location_stores_to_Joann.to_csv("./walmart_stores.csv",index=False)

In [8]:
df_walmart_location

,safegraph_place_id,location_name,brands,street_address,city,region,postal_code,latitude,longitude
921,sg:43f385d5026e4fffb55bbd42f0a91302,Walmart,Walmart,2400 Veterans Memorial Pkwy,Orange City,FL,32763,28.913804,-81.278884
1045,sg:4a8dc2497b1441ca8292e9ed302dd966,Walmart,Walmart,201 Gateway Blvd,Rock Springs,WY,82901,41.580524,-109.251274
2126,sg:9c63817bf6194d4b9e696d06444310ac,Walmart,Walmart,12000 Iron Bridge Rd,Chester,VA,23831,37.350503,-77.482775
2741,sg:cbe350b708b44a02b341ed66b386942f,Walmart,Walmart,4115 Calumet Ave,Manitowoc,WI,54220,44.075956,-87.696784
3303,sg:f86b790b3e98461a9493853531b5e46c,Walmart,Walmart,2310 E Serene Ave,Las Vegas,NV,89123,36.022987,-115.124299
...,...,...,...,...,...,...,...,...,...
532578,sg:2aeb24b60c1144abb891f0c5d77df2a1,Walmart,Walmart,1101 E Prospect Ave,Ponca City,OK,74601,36.741109,-97.071893
534386,sg:ba81871e7110490287334b713d192cd8,Walmart,Walmart,349 Kinston Hwy,Richlands,NC,28574,34.907981,-77.567338
535876,sg:30071dbc2cc041e283c406064c904995,Walmart,Walmart,677 Timpany Blvd,Gardner,MA,01440,42.551510,-71.994626
537429,sg:ac58140ad6634fc487c93b0f9faf2e49,Walmart,Walmart,8450 La Palma Ave,Buena Park,CA,90620,33.844659,-117.986690


In [9]:
df_walmart_location.shape,df_walmart_location['safegraph_place_id'].nunique()

((4693, 9), 4693)

# Zips in X miles from the store

In [10]:
radius=10
zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json","r"))

In [11]:
df_poi_locations=df_walmart_location[['safegraph_place_id','latitude','longitude']].drop_duplicates()
df_poi_locations['coor']=df_poi_locations[['latitude','longitude']].values.tolist()
dict_poi_locations=df_poi_locations.set_index("safegraph_place_id").to_dict()['coor']

In [12]:
df_zip_by_poi=pd.DataFrame()
i_counter=0
for poi_id, poi_coor in dict_poi_locations.items():
    list_zips_in_x_miles=[]
    for zip_cd, zip_centroid in zip_centers.items():
        dist=haversine(poi_coor,zip_centroid,unit="mi")
        if dist<=radius:
            list_zips_in_x_miles.append(zip_cd)
    df=pd.DataFrame({"zip_cd":list_zips_in_x_miles},index=[poi_id]*len(list_zips_in_x_miles)).reset_index().rename(columns={"index":"poi_id"})
    
    df_zip_by_poi=df_zip_by_poi.append(df)
    i_counter+=1
    if i_counter%100==0:
        print(i_counter,datetime.datetime.now())

100 2020-05-29 13:36:40.571940
200 2020-05-29 13:36:54.228635
300 2020-05-29 13:37:07.895100
400 2020-05-29 13:37:21.557821
500 2020-05-29 13:37:35.202868
600 2020-05-29 13:37:48.873460
700 2020-05-29 13:38:02.528935
800 2020-05-29 13:38:16.179645
900 2020-05-29 13:38:29.855066
1000 2020-05-29 13:38:43.538657
1100 2020-05-29 13:38:57.208993
1200 2020-05-29 13:39:10.903111
1300 2020-05-29 13:39:24.597058
1400 2020-05-29 13:39:38.292911
1500 2020-05-29 13:39:52.010562
1600 2020-05-29 13:40:05.711201
1700 2020-05-29 13:40:19.441079
1800 2020-05-29 13:40:33.179001
1900 2020-05-29 13:40:46.897904
2000 2020-05-29 13:41:00.632388
2100 2020-05-29 13:41:14.357401
2200 2020-05-29 13:41:28.117573
2300 2020-05-29 13:41:41.863352
2400 2020-05-29 13:41:55.580060
2500 2020-05-29 13:42:09.308872
2600 2020-05-29 13:42:23.048717
2700 2020-05-29 13:42:36.788183
2800 2020-05-29 13:42:50.547325
2900 2020-05-29 13:43:04.259986
3000 2020-05-29 13:43:17.990350
3100 2020-05-29 13:43:31.740120
3200 2020-05-29 1

In [13]:
print(df_zip_by_poi.shape)
df_zip_by_poi.head(4)

(97329, 2)


,poi_id,zip_cd
0,sg:43f385d5026e4fffb55bbd42f0a91302,32744
1,sg:43f385d5026e4fffb55bbd42f0a91302,32713
2,sg:43f385d5026e4fffb55bbd42f0a91302,32725
3,sg:43f385d5026e4fffb55bbd42f0a91302,32724


In [14]:
df_unique_zip_cd_in_10_miles=df_zip_by_poi[['zip_cd']].drop_duplicates()
df_unique_zip_cd_in_10_miles.shape

(24126, 1)

# Demo by zip_cd

In [15]:
# demo needed:
'''
population
1. age 25-54 (applied the census with the mapping pctg)
2. parents with children<age 18 (from EASI zip 4, single with child hh + married with child hh *2)
'''  

# Demo data
'''
1. zip-4 level EASI (A,B,C 3 parts): "/home/jian/Docs/EASI/2018-07-19/"
2. ZCTA level Census from the factfinder, need to download new if you want something you need: "/home/jian/Docs/Household_and_Population/Cencus_2019/"
3. ZCTA to zip_cd (postal code in 5 digit) mapping: "/home/jian/Docs/Household_and_Population/Cencus_2019/zcta_zip_mapping/zip_to_zcta_2018.xlsx"
'''

'\n1. zip-4 level EASI (A,B,C 3 parts): "/home/jian/Docs/EASI/2018-07-19/"\n2. ZCTA level Census from the factfinder, need to download new if you want something you need: "/home/jian/Docs/Household_and_Population/Cencus_2019/"\n3. ZCTA to zip_cd (postal code in 5 digit) mapping: "/home/jian/Docs/Household_and_Population/Cencus_2019/zcta_zip_mapping/zip_to_zcta_2018.xlsx"\n'

In [16]:
df_EASI_C=pd.read_csv("/home/jian/Docs/EASI/2018-07-19/ZIP4_18_DATA_C2_CSV/ZIP4_18_DATA_C2.CSV",
                      skiprows=1,usecols=['ZIP_CODE','ZIP4','HH18','PCTHHL25','PCTHH25_34','PCTHH35_44','PCTHH45_54','PCTMARC','PCTSPARC'])
df_EASI_C['ZIP_CODE']=df_EASI_C['ZIP_CODE'].astype(str).apply(lambda x: x.zfill(5))
df_EASI_C['ZIP4']=df_EASI_C['ZIP4'].astype(str).apply(lambda x: x.zfill(4))

df_EASI_C['HH_0_25']=df_EASI_C['HH18']*df_EASI_C['PCTHHL25']/100
df_EASI_C['HH_25_34']=df_EASI_C['HH18']*df_EASI_C['PCTHH25_34']/100
df_EASI_C['HH_35_44']=df_EASI_C['HH18']*df_EASI_C['PCTHH35_44']/100
df_EASI_C['HH_45_54']=df_EASI_C['HH18']*df_EASI_C['PCTHH45_54']/100
df_EASI_C['HH_25_54']=df_EASI_C['HH_25_34']+df_EASI_C['HH_35_44']+df_EASI_C['HH_45_54']

df_EASI_C['HH_MARC']=df_EASI_C['HH18']*df_EASI_C['PCTMARC']/100
df_EASI_C['HH_SPARC']=df_EASI_C['HH18']*df_EASI_C['PCTSPARC']/100
df_EASI_C['HH_With_Children']=df_EASI_C['HH_MARC']+df_EASI_C['HH_SPARC']
df_EASI_C['parents_with_Children']=df_EASI_C['HH_MARC']*2+df_EASI_C['HH_SPARC']

df_zip_HH=df_EASI_C.groupby(["ZIP_CODE"])['HH18','HH_With_Children','parents_with_Children'].sum().reset_index()
del df_EASI_C
gc.collect()
df_zip_HH.head(2)

/home/jian/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ZIP_CODE,HH18,HH_With_Children,parents_with_Children
0,00501,0.0,0.00000,0.0000
1,01001,6853.6,1552.99957,2580.7664


In [17]:
df_zip_HH=df_zip_HH.rename(columns={"HH18":"Households"})
df_zip_HH.head(4)

,ZIP_CODE,Households,HH_With_Children,parents_with_Children
0,00501,0.0,0.00000,0.00000
1,01001,6853.6,1552.99957,2580.76640
2,01002,9835.6,2185.14329,3526.27847
3,01003,0.0,0.00000,0.00000


In [18]:
df_zip_ZCTA_mapping=pd.ExcelFile("/home/jian/Docs/Household_and_Population/Cencus_2019/zcta_zip_mapping/zip_to_zcta_2018.xlsx")
print(df_zip_ZCTA_mapping.sheet_names)
df_zip_ZCTA_mapping=df_zip_ZCTA_mapping.parse("ziptozcta2017",dtype=str)
print(df_zip_ZCTA_mapping.shape)
df_zip_ZCTA_mapping.head(2)

df_zip_ZCTA_mapping=df_zip_ZCTA_mapping[['ZIP_CODE',"ZCTA"]].drop_duplicates()
print("shape",df_zip_ZCTA_mapping.shape)
print("ZIP_CODE nunique",df_zip_ZCTA_mapping['ZIP_CODE'].nunique())
print("ZCTA nunique",df_zip_ZCTA_mapping['ZCTA'].nunique())
df_zip_ZCTA_mapping.head(5)

['ziptozcta2017', 'source']
(41160, 6)
shape (41160, 2)
ZIP_CODE nunique 41160
ZCTA nunique 33138


,ZIP_CODE,ZCTA
0,00501,11742
1,00544,11742
2,00601,00601
3,00602,00602
4,00603,00603


In [19]:
census_data_S0101=pd.read_csv("/home/jian/Docs/Household_and_Population/Cencus_2019/ACS_17_5YR_S0101_with_ann.csv",dtype=str,skiprows=1)
print(census_data_S0101.shape)

census_population_needed=census_data_S0101[['Id2','Total; Estimate; Total population',
                                            'Total; Estimate; AGE - 25 to 29 years',
                                            'Total; Estimate; AGE - 30 to 34 years',
                                            'Total; Estimate; AGE - 35 to 39 years',
                                            'Total; Estimate; AGE - 40 to 44 years',
                                            'Total; Estimate; AGE - 45 to 49 years',
                                            'Total; Estimate; AGE - 50 to 54 years',
                                            'Total; Estimate; AGE - Under 5 years',
                                            'Total; Estimate; AGE - 5 to 9 years',
                                            'Total; Estimate; AGE - 10 to 14 years',
                                            'Total; Estimate; AGE - 15 to 19 years',
                                           ]]

for col in census_population_needed.columns.tolist()[1:]:
    census_population_needed[col]=census_population_needed[col].astype(float)
census_population_needed['population_25_54']=census_population_needed[['Total; Estimate; AGE - 25 to 29 years',
                                            'Total; Estimate; AGE - 30 to 34 years',
                                            'Total; Estimate; AGE - 35 to 39 years',
                                            'Total; Estimate; AGE - 40 to 44 years',
                                            'Total; Estimate; AGE - 45 to 49 years',
                                            'Total; Estimate; AGE - 50 to 54 years']].sum(axis=1)
census_population_needed['population_under_19']=census_population_needed[['Total; Estimate; AGE - Under 5 years',
                                            'Total; Estimate; AGE - 5 to 9 years',
                                            'Total; Estimate; AGE - 10 to 14 years',
                                            'Total; Estimate; AGE - 15 to 19 years']].sum(axis=1)

census_population_needed=census_population_needed.rename(columns={"Id2":"ZCTA","Total; Estimate; Total population":"total_population"})
census_population_needed=census_population_needed[['ZCTA','total_population','population_under_19','population_25_54']]
census_population_needed.head(3)

(33120, 459)


/home/jian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.or

,ZCTA,total_population,population_under_19,population_25_54
0,00601,17599.0,4512.0,6664.0
1,00602,39209.0,8933.0,15520.0
2,00603,50135.0,11950.0,19086.0


In [20]:
census_population_needed.head(2)

,ZCTA,total_population,population_under_19,population_25_54
0,00601,17599.0,4512.0,6664.0
1,00602,39209.0,8933.0,15520.0


In [21]:
df_census=pd.merge(df_zip_ZCTA_mapping,census_population_needed,on="ZCTA",how="outer")
df_census_by_ZCTA=df_census.groupby(['ZCTA'])['total_population'].sum().to_frame().reset_index()
df_census_by_ZCTA=df_census_by_ZCTA.rename(columns={"total_population":"ZCTA_level_pop"})
df_census=pd.merge(df_census,df_census_by_ZCTA,on="ZCTA",how="left")
print(df_census[pd.isnull(df_census['ZIP_CODE'])].shape)
df_census=df_census[pd.notnull(df_census['ZIP_CODE'])]

(6, 6)


In [22]:
print(df_zip_HH.shape,df_zip_HH['ZIP_CODE'].nunique())
print(df_census.shape,df_census['ZIP_CODE'].nunique())

(39602, 4) 39602
(41160, 6) 41160


In [93]:
df_output=pd.merge(df_zip_HH,df_census,on="ZIP_CODE",how="left")
df_output['zip_cd_pctg_of_ZCTA']=df_output['total_population']/df_output['ZCTA_level_pop']
df_output['population_under_19_applied']=df_output['population_under_19']*df_output['zip_cd_pctg_of_ZCTA']
df_output['population_25_54_applied']=df_output['population_25_54']*df_output['zip_cd_pctg_of_ZCTA']

df_output['total_population_applied']=df_output['total_population']*df_output['zip_cd_pctg_of_ZCTA']

In [96]:
df_output=df_output[['ZIP_CODE','Households','HH_With_Children','parents_with_Children','population_under_19_applied',
                     'total_population_applied','population_25_54_applied']]

In [97]:
df_output.head(3)

,ZIP_CODE,Households,HH_With_Children,parents_with_Children,population_under_19_applied,total_population_applied,population_25_54_applied
0,00501,0.0,0.00000,0.00000,1102.333333,4386.000000,1934.000000
1,01001,6853.6,1552.99957,2580.76640,3400.000000,17537.000000,7023.000000
2,01002,9835.6,2185.14329,3526.27847,2415.333333,10093.333333,2577.666667


# IRI

In [102]:
df_tmp=pd.read_excel("/home/jian/Docs/Geo_mapping/Total_US_counties_IRImapping_markets_and_regions_cr_02192019.xlsx",
                            skiprows=1,dtype=str)
df_tmp.head()

,Fipscode,County Name,State,IRI Market,IRI Region,Unnamed: 5,Client Releasable
0,1001,AUTAUGA,AL,BIRMINGHAM/MONTGOMERY,SOUTHEAST,NaN,NaN
1,1003,BALDWIN,AL,NEW ORLEANS/MOBILE,SOUTHEAST,NaN,NaN
2,1005,BARBOUR,AL,NO MARKET,SOUTHEAST,NaN,NaN
3,1007,BIBB,AL,BIRMINGHAM/MONTGOMERY,SOUTHEAST,NaN,NaN
4,1009,BLOUNT,AL,BIRMINGHAM/MONTGOMERY,SOUTHEAST,NaN,NaN


In [26]:
df_IRI_county=pd.read_excel("/home/jian/Docs/Geo_mapping/Total_US_counties_IRImapping_markets_and_regions_cr_02192019.xlsx",
                            skiprows=1,dtype=str,usecols=['County Name','State','IRI Market'])
df_IRI_county=df_IRI_county.drop_duplicates()
print(df_IRI_county.shape,df_IRI_county['IRI Market'].nunique())

(3110, 3) 66


In [27]:
df_IRI_county['IRI_county']=df_IRI_county['County Name']+", "+df_IRI_county['State']
df_IRI_county['IRI_county']=df_IRI_county['IRI_county'].apply(lambda x: x.lower().strip())
df_IRI_county=df_IRI_county[['IRI Market','IRI_county']].reset_index()
del df_IRI_county['index']

In [28]:
df_DMA_zip_county=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",
                                dtype=str,skiprows=1,usecols=[0,6,9])
df_DMA_zip_county.columns=['zip_cd','ST','CTY']
df_DMA_zip_county['Nielsen_county']=df_DMA_zip_county['CTY']+", "+df_DMA_zip_county['ST']
df_DMA_zip_county['Nielsen_county']=df_DMA_zip_county['Nielsen_county'].apply(lambda x: x.lower().strip())
df_DMA_zip_county=df_DMA_zip_county[['zip_cd','Nielsen_county']].drop_duplicates()
df_DMA_zip_county.shape

(51324, 2)

In [29]:
list_nielsen_county=df_DMA_zip_county['Nielsen_county'].unique().tolist()

In [30]:
from fuzzywuzzy import fuzz

df_iri_matching_counties=df_IRI_county.copy()
df_iri_matching_counties['matched_nielsen_county']=np.nan
df_iri_matching_counties['fuzz_ratio']=np.nan
i_counter=0
for i,row in df_iri_matching_counties.iterrows():
    i_counter+=1
    iri_name=row['IRI_county']
    best_score=0
    best_match=np.nan
    
    for nielsen_name in list_nielsen_county:
        score=fuzz.ratio(iri_name,nielsen_name) 
        if score>=best_score:
            best_score=score
            best_match=nielsen_name
    df_iri_matching_counties.loc[i,'matched_nielsen_county']=best_match
    df_iri_matching_counties.loc[i,'fuzz_ratio']=best_score
    if i_counter%100==1:
        print(i_counter,datetime.datetime.now())

/usr/local/lib/python3.6/site-packages/fuzzywuzzy/fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


1 2020-05-29 13:49:29.812090
101 2020-05-29 13:49:44.385895
201 2020-05-29 13:49:59.348505
301 2020-05-29 13:50:14.018556
401 2020-05-29 13:50:28.534422
501 2020-05-29 13:50:43.017929
601 2020-05-29 13:50:57.439759
701 2020-05-29 13:51:11.952935
801 2020-05-29 13:51:26.507784
901 2020-05-29 13:51:40.920806
1001 2020-05-29 13:51:55.200687
1101 2020-05-29 13:52:09.861051
1201 2020-05-29 13:52:25.530475
1301 2020-05-29 13:52:40.307221
1401 2020-05-29 13:52:55.071833
1501 2020-05-29 13:53:09.470508
1601 2020-05-29 13:53:24.193296
1701 2020-05-29 13:53:38.522161
1801 2020-05-29 13:53:53.147242
1901 2020-05-29 13:54:07.995814
2001 2020-05-29 13:54:22.503051
2101 2020-05-29 13:54:36.867491
2201 2020-05-29 13:54:51.309472
2301 2020-05-29 13:55:06.517151
2401 2020-05-29 13:55:20.959355
2501 2020-05-29 13:55:35.519557
2601 2020-05-29 13:55:49.720058
2701 2020-05-29 13:56:03.981262
2801 2020-05-29 13:56:18.380428
2901 2020-05-29 13:56:34.264081
3001 2020-05-29 13:56:49.249259
3101 2020-05-29 13:5

In [31]:
pd.options.display.max_rows=100
df_iri_matching_counties[df_iri_matching_counties['fuzz_ratio']<=99]

,IRI Market,IRI_county,matched_nielsen_county,fuzz_ratio
24,BIRMINGHAM/MONTGOMERY,"de kalb, al","dekalb, al",95.0
67,NO MARKET,"apache, az","apache-n, az",91.0
165,SACRAMENTO,"el dorado, ca","el dorado-e, ca",93.0
171,NO MARKET,"kern, ca","kern-e, ca",89.0
189,LOS ANGELES,"riverside, ca","riverside-e, ca",93.0
192,LOS ANGELES,"san bernardino, ca","san bernardno, ca",97.0
196,NO MARKET,"san luis obispo, ca","san luis obpo, ca",94.0
198,NO MARKET,"santa barbara, ca","santa brbra-n, ca",88.0
204,NO MARKET,"solano, ca","solano-w, ca",91.0
289,BALTIMORE/WASHINGTON,"dist of columbia, dc","dis of col, dc",82.0


In [32]:
list_looks_no_good_iri_counties=['west feliciana, la','shannon, sd','alexandria, va',
                                 'clifton forge, va','colonial heights, va','danville, va',
                                 'emporia, va','falls church, va','galax, va','harrisonburg, va',
                                'hopewell, va','lynchburg, va','manassas, va','manassas park, va',
                                'martinsville, va','norfolk, va','norton, va','petersburg, va',
                                'poquoson, va','radford, va','salem, va','south boston, va',
                                 'staunton, va','suffolk, va','williamsburg, va','winchester, va']
print(len(list_looks_no_good_iri_counties))

26


In [33]:
'''
df_to_fill=pd.DataFrame({"iri_county":list_looks_no_good_iri_counties})
df_to_fill['nielsen_county']=np.nan
df_to_fill.to_csv("./to_eyebow_the_matching_Nielsen_county.csv",index=False)
'''

'\ndf_to_fill=pd.DataFrame({"iri_county":list_looks_no_good_iri_counties})\ndf_to_fill[\'nielsen_county\']=np.nan\ndf_to_fill.to_csv("./to_eyebow_the_matching_Nielsen_county.csv",index=False)\n'

In [34]:
df_rematched_iri_county=pd.read_excel("/home/jian/Docs/POI/Application_of_the_POI/renamed_county_VA_part.xlsx",dtype=str,sheet_name="to_eyebow_the_matching_Nielsen_")
df_rematched_iri_county['nielsen_county']=df_rematched_iri_county['nielsen_county'].str.lower()
dict_rematched_iri_county=df_rematched_iri_county.set_index("iri_county").to_dict()["nielsen_county"]

In [35]:
df_rematched_iri_county.head(2)

,iri_county,nielsen_county
0,"west feliciana, la","w feliciana, la"
1,"shannon, sd","oglala lakota, sd"


In [36]:
df_mapping_counties=df_iri_matching_counties[['IRI_county','matched_nielsen_county']].drop_duplicates()
print("df_mapping_counties.shape",df_mapping_counties.shape)
df_mapping_counties_1=df_mapping_counties[~df_mapping_counties['IRI_county'].isin(list_looks_no_good_iri_counties)]
df_mapping_counties_2=df_mapping_counties[df_mapping_counties['IRI_county'].isin(list_looks_no_good_iri_counties)]
df_mapping_counties_2['matched_nielsen_county']=df_mapping_counties_2['IRI_county'].apply(lambda x: dict_rematched_iri_county[x])
print("df_mapping_counties_1.shape",df_mapping_counties_1.shape)
print("df_mapping_counties_2.shape",df_mapping_counties_2.shape)
df_mapping_counties=df_mapping_counties_1.append(df_mapping_counties_2).reset_index()
del df_mapping_counties['index']
print("df_mapping_counties.shape",df_mapping_counties.shape)


df_mapping_counties.shape (3110, 2)
df_mapping_counties_1.shape (3084, 2)
df_mapping_counties_2.shape (26, 2)
df_mapping_counties.shape (3110, 2)


/home/jian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [37]:
df_mapping_counties.head(5)

,IRI_county,matched_nielsen_county
0,"autauga, al","autauga, al"
1,"baldwin, al","baldwin, al"
2,"barbour, al","barbour, al"
3,"bibb, al","bibb, al"
4,"blount, al","blount, al"


In [38]:
df_DMA_zip_county=pd.merge(df_DMA_zip_county,df_mapping_counties,left_on="Nielsen_county",right_on="matched_nielsen_county",how="outer")
df_DMA_zip_county=pd.merge(df_DMA_zip_county,df_IRI_county,on="IRI_county",how="outer")
print(df_DMA_zip_county.shape)
df_DMA_zip_county.head(2)

(52287, 5)


,zip_cd,Nielsen_county,IRI_county,matched_nielsen_county,IRI Market
0,00501,"suffolk, ny","suffolk, ny","suffolk, ny",NEW YORK
1,00544,"suffolk, ny","suffolk, ny","suffolk, ny",NEW YORK


In [39]:
df_county_only_in_Nielsen=df_DMA_zip_county[pd.isnull(df_DMA_zip_county['IRI Market'])]['Nielsen_county'].drop_duplicates().reset_index()
del df_county_only_in_Nielsen['index']
df_county_only_in_Nielsen.head(3)

,Nielsen_county
0,"oneida-w, ny"
1,"monroe-rem, ny"
2,"rochestr city, ny"


In [40]:
list_iri_counties=df_IRI_county['IRI_county'].unique().tolist()

df_county_only_in_Nielsen['matched_iri_county']=np.nan
df_county_only_in_Nielsen['fuzz_ratio']=np.nan
i_counter=0
for i,row in df_county_only_in_Nielsen.iterrows():
    i_counter+=1
    nielsen_name=row['Nielsen_county']
    best_score=0
    best_match=np.nan
    
    for iri_name in list_iri_counties:
        score=fuzz.ratio(nielsen_name,iri_name) 
        if score>=best_score:
            best_score=score
            best_match=iri_name
    df_county_only_in_Nielsen.loc[i,'matched_iri_county']=best_match
    df_county_only_in_Nielsen.loc[i,'fuzz_ratio']=best_score

        

In [41]:
list_no_good_match_nielsen=['rochestr city, ny','honolulu, hi', 'kauai, hi','hawaii, hi','maui, hi',
                            'anchorage, ak','kenai-pensla, ak','matanka-sustn, ak','fairbnks-plus, ak','juneau-plus, ak']
# manually hard coded
# Google MONROE,NY for rochestr city, and all AK & HI no market
dict_no_good_match_in_nielsen=dict({
    'rochestr city, ny':"monroe, ny",
    'honolulu, hi':"no market", 
    'kauai, hi':"no market", 
    'hawaii, hi':"no market", 
    'maui, hi':"no market", 
    'anchorage, ak':"no market", 
    'kenai-pensla, ak':"no market", 
    'matanka-sustn, ak':"no market", 
    'fairbnks-plus, ak':"no market", 
    'juneau-plus, ak':"no market"
})

df_df_county_only_in_Nielsen_1=df_county_only_in_Nielsen[~df_county_only_in_Nielsen['Nielsen_county'].isin(list_no_good_match_nielsen)]
df_df_county_only_in_Nielsen_2=df_county_only_in_Nielsen[df_county_only_in_Nielsen['Nielsen_county'].isin(list_no_good_match_nielsen)]
df_df_county_only_in_Nielsen_2['matched_iri_county']=df_df_county_only_in_Nielsen_2['Nielsen_county'].apply(lambda x: dict_no_good_match_in_nielsen[x])

df_county_only_in_Nielsen=df_df_county_only_in_Nielsen_1.append(df_df_county_only_in_Nielsen_2)
del df_county_only_in_Nielsen['fuzz_ratio']
dict_county_only_in_Nielsen=df_county_only_in_Nielsen.set_index("Nielsen_county").to_dict()['matched_iri_county']

/home/jian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [42]:
df_1=df_DMA_zip_county[pd.notnull(df_DMA_zip_county['IRI Market'])]
df_2=df_DMA_zip_county[pd.isnull(df_DMA_zip_county['IRI Market'])]


In [43]:
df_2['IRI_county']=df_2['Nielsen_county'].apply(lambda x: dict_county_only_in_Nielsen[x])
del df_2['IRI Market']
df_2=pd.merge(df_2,df_IRI_county,on="IRI_county",how="left")
df_DMA_zip_county=df_1.append(df_2)
df_DMA_zip_county=df_DMA_zip_county[['zip_cd','Nielsen_county','IRI_county','IRI Market']].drop_duplicates()

/home/jian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [44]:
# df_DMA_zip_county.to_csv("./zip_county_IRI_mapping_JL_"+str(datetime.datetime.now().date())+".csv",index=False)

In [45]:
df_IRI=df_DMA_zip_county[['zip_cd','IRI Market']].drop_duplicates()
df_IRI=df_IRI.groupby("zip_cd")['IRI Market'].apply(list).reset_index()
df_IRI['len_iri_mkt']=df_IRI['IRI Market'].apply(len)
df_IRI['len_iri_mkt'].unique()

array([1, 2, 3])

In [46]:
df_output=df_output.rename(columns={"ZIP_CODE":"zip_cd"})
df_output=pd.merge(df_output,df_IRI,on="zip_cd",how="left")
df_unique_zip_cd_in_10_miles=pd.merge(df_unique_zip_cd_in_10_miles,df_IRI,on="zip_cd",how="left")

In [64]:
def removenomkt(inputmktlist):
    if type(inputmktlist)==float:
        return inputmktlist
    
    lcopy=inputmktlist.copy()
    
    if len(lcopy)>1:
        try:
            lcopy.remove('NO MARKET')
        except:
            pass
        return lcopy
    else:
        return lcopy

In [81]:
df_unique_zip_cd_in_10_miles['iri_removed_nomkt_if_multi']=df_unique_zip_cd_in_10_miles['IRI Market'].apply(lambda x: removenomkt(x))
df_unique_zip_cd_in_10_miles['len_iri_removed_nomkt']=df_unique_zip_cd_in_10_miles['iri_removed_nomkt_if_multi'].apply(lambda x: len(x) if type(x)!=float else x)

In [87]:

df_zips_with_dup_rows=df_unique_zip_cd_in_10_miles[['zip_cd','iri_removed_nomkt_if_multi']]
print(df_zips_with_dup_rows.shape)
'''
df_zips_with_dup_rows['iri_removed_nomkt_if_multi']=df_zips_with_dup_rows['iri_removed_nomkt_if_multi'].astype(str)
df_zips_with_dup_rows=df_zips_with_dup_rows.drop_duplicates()
print(df_zips_with_dup_rows.shape)
'''
# all unique

(24126, 2)


"\ndf_zips_with_dup_rows['iri_removed_nomkt_if_multi']=df_zips_with_dup_rows['iri_removed_nomkt_if_multi'].astype(str)\ndf_zips_with_dup_rows=df_zips_with_dup_rows.drop_duplicates()\nprint(df_zips_with_dup_rows.shape)\n"

In [90]:
df_zips_with_dup_rows=df_zips_with_dup_rows.reset_index()
del df_zips_with_dup_rows['index']
df_zips_with_dup_rows_output=pd.DataFrame()
for i, row in df_zips_with_dup_rows.iterrows():
    if type(row['iri_removed_nomkt_if_multi'])!=float:
        df=pd.DataFrame({"iri_mkt":row['iri_removed_nomkt_if_multi']},index=[row['zip_cd']]*len(row['iri_removed_nomkt_if_multi']))
    else:
        df=pd.DataFrame({"iri_mkt":row['iri_removed_nomkt_if_multi']},index=[row['zip_cd']])
    df=df.reset_index().rename(columns={"index":"zip_cd"})
    df_zips_with_dup_rows_output=df_zips_with_dup_rows_output.append(df)

In [91]:
df_zips_with_dup_rows_output.shape

(24304, 2)

In [98]:
writer=pd.ExcelWriter("./To_Joann_US_zips_selected_demo_by_zip_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_output.to_excel(writer,"based_on_EASI_and_census",index=False)
df_unique_zip_cd_in_10_miles.to_excel(writer,"Walmart_10_miles",index=False)
df_zips_with_dup_rows_output.to_excel(writer,"zip_walmart_dup_rows",index=False)
writer.save()